# Importing Data from MongoDB

In [2]:
import pandas as pd
import numpy as np
from pymongo import MongoClient
import matplotlib.pyplot as plt

In [3]:
client = MongoClient('localhost', 27017)

In [4]:
db = client.default
collection = db.game_data

In [5]:
cursor = collection.find({})

In [6]:
documents = []
for cnt in range(collection.count()):
    documents.append(next(cursor))

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  


In [9]:
data = pd.DataFrame(documents)

In [11]:
data = data[data.monster_logs.map(len)>=1]

In [18]:
##### 용 개수 0개인거 예외 처리 해야됨 - 용 0개인거 다 지우는 절 추가

def preprocessData1(dataset):
    index = len(dataset)
    df = pd.DataFrame(index= list(range(1,index+1)), columns = ['time'])
    df['time'] = dataset['game_duration'].tolist()
    t1 = dataset['monster_logs'].apply(lambda x : [x[i].get('monster_type') for i in range(len(x))]).tolist()
    objlist = simplify(t1)
    t2 = dataset['teams'].apply(lambda x : x['win']['participants'][0]['stats']['participantId']).tolist()
    t3 = dataset['monster_logs'].apply(lambda x : [x[i].get('killer_id') for i in range(len(x))]).tolist()
    winlist = winnerObj(t2,t3)
    x = drakelist(objlist,winlist)[0]
    y = drakelist(objlist,winlist)[1]
    df['drake_sequence'] = x
    a = winobjlist(x,y)
    b = loseobjlist(x,y)
    f = lambda x : x.count("E")
    df['winning_team_elder_drakes'] = [f(x) for x in a]
    df['losing_team_elder_drakes'] = [f(x) for x in b]
    df['winning_team_elemental_drakes'] = dragonsoul(a,b)[2]
    df['losing_team_elemental_drakes'] = dragonsoul(a,b)[3]
    df['soul'] = dragonsoul(a,b)[0]
    df['winning_team_has_soul'] = dragonsoul(a,b)[1]
    df['winning_team_first_dragon'] = dataset['teams'].apply(lambda x : x['win']['first_dragon']).tolist()
    t4 = dataset['monster_logs'].apply(lambda x : [x[i].get('timestamp') for i in range(len(x))]).tolist()
    draketimelist = draketime(objlist, t4)
    df['first_drake_time'] = [x[0] if len(x) != 0 else "No Drakes" for x in draketimelist]
    df['last_drake_time'] = [x[-1] if len(x) != 0 else "No Drakes" for x in draketimelist]
    ##winning_team_last_dragon에 문제 있음 : 수정 필요
    df['winning_team_last_dragon'] = drakelist(objlist,winlist)[2]
    df['tier'] = dataset['tier'].tolist()
    #df['first_drake_time'] = dataset['monster_logs'].apply(lambda x : [x[i].get('timestamp') for i in range(len(x))]).tolist()
    #df['win_team_jg_champ'] = dataset['teams'].apply(lambda x : x['win']['participants'][0]['stats']['participantId']).tolist()
    #df['other_obj_sequence'] = otherlist(objlist,winlist)[0]
    return df

def simplify(objlist):
    p = []
    for i in range(len(objlist)):
        q = ''
        for j in range(len(objlist[i])):
            if objlist[i][j] == 'EARTH_DRAGON':
                q += "M"
            else : 
                q += str(objlist[i][j][0])
        p.append(q)
    return p

def winnerObj(winteam, killer_id):
    index = [1,2,3,4,5]
    b = []
    
    for i in range(len(killer_id)):
        if len(killer_id[i]) == 0:
            b.append('')
        else:
            a = ''
            for j in range(len(killer_id[i])):
                if killer_id[i][j] == 0 :
                    a += 'N'
                elif (killer_id[i][j] in index) ^ (winteam[i] in index) :
                    a += 'L'
                else : 
                    a += 'W'
            b.append(a)
    return b
        
def findOccurrences(s, objlist):
    return [i for i, letter in enumerate(s) if letter in objlist]

def drakelist(objlist, winlist):
    drake = ['A','W','F','E','M']
    drakelist = []
    drakewinlist = []
    lastdrakewin = []
    for i in range(len(objlist)):
        index = findOccurrences(objlist[i], drake)
        n1 = ''
        n2 = ''
        for j in range(len(index)):
            n1 += objlist[i][index[j]]
            n2 += winlist[i][index[j]]
        drakelist.append(n1)
        drakewinlist.append(n2)
        if len(n2) != 0:
            lastdrakewin.append(n2[-1] == 'W')
        else :
            lastdrakewin.append('No Drake')
    return drakelist, drakewinlist, lastdrakewin

def draketime(objlist, timelist):
    drake = ['A','W','F','E','M']
    draketimelist = []
    for i in range(len(objlist)):
        index = findOccurrences(objlist[i], drake)
        n2 = []
        for j in range(len(index)):
            n2.append(timelist[i][index[j]])
        draketimelist.append(n2)
    return draketimelist
    
def otherlist(objlist, winlist):
    other = ['R','B']
    otherlist = []
    otherwinlist = []
    for i in range(len(objlist)):
        index = findOccurrences(objlist[i], other)
        n1 = ''
        n2 = ''
        for j in range(len(index)):
            if winlist[i][index[j]] != 'N':
                n1 += objlist[i][index[j]]
                n2 += winlist[i][index[j]]
        otherlist.append(n1)
        otherwinlist.append(n2)
    return otherlist, otherwinlist

def winobjlist(objlist, winlist):
    winobjlist = []
    for i in range(len(objlist)):
        index = findOccurrences(winlist[i], 'W')
        n1 = ''
        for j in range(len(index)):
            n1 += objlist[i][index[j]]
        winobjlist.append(n1)
    return winobjlist
    
def loseobjlist(objlist, winlist):
    loseobjlist = []
    for i in range(len(objlist)):
        index = findOccurrences(winlist[i], 'L')
        n1 = ''
        for j in range(len(index)):
            n1 += objlist[i][index[j]]
        loseobjlist.append(n1)
    return loseobjlist

def dragonsoul(windrake, losedrake):
    p = []
    q = []
    elemental_drake = ['A','W','F','M']
    windrake = [c.replace('E', '') for c in windrake]
    losedrake = [c.replace('E', '') for c in losedrake]
    for i in range(len(windrake)):
        if max(len(windrake[i]), len(losedrake[i])) < 4:
            p.append("No Soul")
            q.append("N/A")
        elif len(windrake[i]) == 4:
            p.append(windrake[i][3])
            q.append("True")
        elif len(losedrake[i]) == 4: #else?
            p.append(losedrake[i][3])
            q.append("False")
    return p,q, windrake, losedrake

In [28]:
final = preprocessData1(data)
final['drake_sequence'].replace('', np.nan, inplace=True)
final.dropna(subset = ['drake_sequence'], inplace=True)
final

,time,drake_sequence,winning_team_elder_drakes,losing_team_elder_drakes,winning_team_elemental_drakes,losing_team_elemental_drakes,soul,winning_team_has_soul,winning_team_first_dragon,first_drake_time,last_drake_time,winning_team_last_dragon,tier
1,680,A,0,0,A,,No Soul,N/A,True,405953,405953,True,MASTER
2,758,W,0,0,W,,No Soul,N/A,True,565683,565683,True,MASTER
3,954,A,0,0,A,,No Soul,N/A,True,675609,675609,True,MASTER
4,946,WA,0,0,WA,,No Soul,N/A,True,543818,863543,True,MASTER
5,1171,AWM,0,0,AWM,,No Soul,N/A,True,398391,1064247,True,MASTER
...,...,...,...,...,...,...,...,...,...,...,...,...,...
758568,2380,FMAAAA,0,0,FAA,MAA,No Soul,N/A,True,552215,2287570,True,GOLD
758569,1976,FMAA,0,0,FMAA,,A,True,True,587081,1547242,True,DIAMOND
758570,1579,FWM,0,0,FWM,,No Soul,N/A,True,628123,1285392,True,SILVER
758571,1303,FWA,0,0,FWA,,No Soul,N/A,True,560124,1291972,True,GOLD


In [26]:
final.tail(50)
final.to_csv("drake_data.csv")